In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from langchain.agents import initialize_agent, AgentType, Tool
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
from utils import load_json
import os
from LegalPrincipleExtractorTool import LegalPrincipleExtractor
from CitationExtractorTool import CitationExtractor
import dotenv
import os
import time
from tqdm import tqdm
from utils import load_json, save_json
dotenv.load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [8]:


data_path = '/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/'
df1_path = '/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_1.csv'
df4_path = '/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_4.csv'
df1 = pd.read_csv(df1_path)['file_path'].to_list()
df4 = pd.read_csv(df4_path)['file_path'].to_list()

case_path = data_path + df4[0]
case = load_json(case_path)



In [9]:
from LegalPrincipleExtractorTool import LegalPrincipleExtractor
from CitationExtractorTool import CitationExtractor
from legalHelperTool import LegalHelper
from FactsReaderTool import CaseFactsReader
from LawReaderTool import CaseLawReader


llm = ChatOpenAI(
        temperature=0,
        model_name="gpt-4o-mini",
        openai_api_key=openai_api_key
    )
    

tools = [
        Tool(
            name='facts_reader',
            func=CaseFactsReader()._run,
            description="Load and return the case facts."
        ),
        Tool(
            name='law_reader',
            func=CaseLawReader()._run,
            description="Load and return the case law."
        ),
        Tool(
            name='legal_principle_extractor',
            func=LegalPrincipleExtractor()._run,
            description="Extract key legal principles from the case file."
        ),
        Tool(
            name='citation_extractor',
            func=CitationExtractor()._run,
            description="Extract citations from the case file."
        ),
        Tool(
            name='legal_helper',
            func=LegalHelper()._run,
            description="""A tool to answer specific legal questions about ECHR jurisprudence and legal principles. 
    Use this tool to ask focused questions about legal concepts, doctrines, or interpretations that will help  analyze the case. 
    The tool do not have access to the case file.
    Do NOT ask this tool to determine if a case is key - instead ask specific questions  about legal elements you identify from the principles and citations.
    To use this tool you must provide [file_path, question] as input.
    """
        )
    ]


    # Simplified prompt - no need to handle text passing
    # Construct the ReAct agent
    # Get the prompt to use - you can modify this!
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools,prompt)


/Users/ahmed/hackatum24/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [15]:
def get_prompt(case_path):
    prompt = f"""
You are a specialized ECHR legal expert tasked with determining if a case is a key case in ECHR jurisprudence. Your role is to conduct an exhaustive legal analysis, akin to writing a detailed case commentary.
Your task is to classify the case at {case_path} into one of the following categories:

KEY CASE:
- Makes a significant contribution to the development, clarification, or modification of case law.
- Establishes new legal principles or substantially modifies existing ones.
- Has broad implications beyond the immediate case.

NOT KEY CASE:
- Applies existing case law without significant contributions to legal development.
- Demonstrates limited implications beyond the immediate dispute.


ANALYSIS WORKFLOW:

-Extract Case Facts and Law
    - Load the case facts and law from the file to understand the context and background.

- Identify Core Legal Issues
    - Determine the key legal questions and associated ECHR articles raised in the case.

- Extract and Analyze Legal Principles
    - Use the legal_principle_extractor tool to extract key principles.
    - Evaluate whether these principles:
        - Develop or clarify existing legal doctrines.
        - Balance competing rights or obligations in novel ways.

- Analyze Cited Precedents
    - Use the citation_extractor tool to extract citations.
    - Investigate:
        - How the case builds upon, clarifies, or diverges from cited precedents.
        - Whether it resolves ambiguities or tensions in prior case law.

- Investigate Novelty
    - Use the extracted principles and citations to determine:
        - Whether the case introduces new legal principles or interpretations.
        - Whether it refines or extends existing doctrines in meaningful ways.

- Assess Precedential Impact
    - Evaluate the case’s potential influence on future jurisprudence:
        -  Does it provide new guidance for interpreting ECHR articles?
        - How frequently is it cited (or could be cited) in subsequent cases?

- Compare with Similar Cases
    - Compare the case to other landmark judgments in the same area of law to assess its distinct contribution (if any).

- Evaluate Broader Implications
    - Consider the case's practical impact:
        - Does it affect state obligations, human rights practices, or jurisprudence in a meaningful way?
        - Does it resolve issues prominent at the time of its ruling?

- Iterative Questioning
    - After each step, ask follow-up questions to deepen your analysis. Examples include:
        - "What ambiguities in cited cases does this case address?"
        - "Does this case set new thresholds for interpreting ECHR rights?"
        - "What practical implications does this case have for future disputes?"

Decision Support Framework
- Summarize findings using a justification grid:
    - Novelty: Identify specific legal developments or clarifications.
    - Practical Impact: Evaluate its influence on future cases or legal practice.
    - Precedential Role: Assess how the case relates to and builds upon prior jurisprudence.

    
Decision and Explanation
- Once your analysis is complete, classify the case as KEY CASE or NOT KEY CASE based on the following guidelines:
- If it IS a Key Case:
    - Identify the specific, novel legal development introduced.
    - Explain why existing jurisprudence was insufficient to address the legal issues.
    - Describe precisely how this case advances ECHR law.

- If it is NOT a Key Case:
    - Explain why the case falls within existing jurisprudence.
    - Highlight its lack of novel contributions, limited implications, or fact-specific nature.   

Important Notes
- Be Rigorous: Each step should inform deeper follow-up questions. Avoid surface-level analysis.
- Focus on Legal Development: Simply applying established principles to new facts does NOT make a case key.
- Avoid Overgeneralization: A case involving multiple articles or issues is NOT automatically key; assess its broader impact and novelty.

Your goal is to produce a comprehensive, systematic, and well-justified analysis of the case.

Your Final answer should follow the format:
- Classifiation: KEY CASE or NOT KEY CASE
- Justification: Detailed explanation of your classification based on the analysis conducted.
    """
    return prompt


#/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-220960.json

prompt = get_prompt(case_path)
agent_executor.invoke({"input": prompt})



> Entering new AgentExecutor chain...
I need to begin by extracting the case facts and law from the provided file to understand the context and background of the case. This will help me identify the core legal issues and the relevant ECHR articles involved.

Action: facts_reader  
Action Input: /Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-229412.json  3. The list of applicants and the relevant details of the applications are set out in the appended table.
4. The applicants complained of the unlawful detention (deprivation of liberty). Some applicants also raised other complaints under the provisions of the Convention or Protocols to it.I have extracted the case facts, which indicate that the applicants are complaining about unlawful detention and possibly other complaints under the ECHR provisions. This gives me a foundational understanding of the case context.

Next, I need to extract the legal law from the case file to understand the legal framework and arguments that are be

KeyboardInterrupt: 

In [13]:
result = agent_executor.invoke({"input": prompt})



> Entering new AgentExecutor chain...
I need to begin by extracting the case facts and law from the provided file to understand the context and background of the case. This will help me identify the core legal issues and the relevant ECHR articles involved.

Action: facts_reader  
Action Input: /Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-229412.json  3. The list of applicants and the relevant details of the applications are set out in the appended table.
4. The applicants complained of the unlawful detention (deprivation of liberty). Some applicants also raised other complaints under the provisions of the Convention or Protocols to it.I have extracted the case facts, which indicate that the applicants are complaining about unlawful detention and possibly other complaints under the ECHR provisions. This gives me a foundational understanding of the case context. Next, I need to extract the relevant law to see how the legal framework applies to these facts.

Action: law_reader  

In [14]:
result

{'input': '\nYou are a specialized ECHR legal expert tasked with determining if a case is a key case in ECHR jurisprudence. Your role is to conduct an exhaustive legal analysis, akin to writing a detailed case commentary.\nYour task is to classify the case at /Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-229412.json into one of the following categories:\n\nKEY CASE:\n- Makes a significant contribution to the development, clarification, or modification of case law.\n- Establishes new legal principles or substantially modifies existing ones.\n- Has broad implications beyond the immediate case.\n\nNOT KEY CASE:\n- Applies existing case law without significant contributions to legal development.\n- Demonstrates limited implications beyond the immediate dispute.\n\n\nANALYSIS WORKFLOW:\n\n-Extract Case Facts and Law\n    - Load the case facts and law from the file to understand the context and background.\n\n- Identify Core Legal Issues\n    - Determine the key legal questions and ass

In [24]:
data_path = '/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/'
df1_path = '/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_1.csv'
df4_path = '/Users/ahmed/Desktop/msc-24/TND/kc_classification_data/pre_cutoff_data/df_4.csv'
df1 = pd.read_csv(df1_path)['file_path'].to_list()
df4 = pd.read_csv(df4_path)['file_path'].to_list()

case_path = data_path + df1[45]

#/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-220960.json

prompt = get_prompt(case_path)
agent_executor.invoke({"input": prompt})



> Entering new AgentExecutor chain...
I need to begin by extracting the case facts and law from the provided file to understand the context and background of the case. This will help me identify the core legal issues and the relevant ECHR articles involved.

Action: facts_reader  
Action Input: /Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-202621.json  1. The applicants were born in 1981 and 1978 respectively and live as a couple in R. They were represented by Ms Diana-Elena Dragomir, a lawyer practising in Bucharest.
2. The Government were represented by their Agent, most recently Ms O.F. Ezer, of the Ministry of Foreign Affairs.
3. The facts of the case, as submitted by the parties, may be summarised as follows.
BACKGROUND OF THE CASE
BACKGROUND OF THE CASE
4. In 2012 the anti-corruption prosecutor s office attached to the High Court of Cassation and Justice ( the prosecutor s office and the HCCJ ) started a criminal investigation into alleged acts of corruption committed by 

{'input': '\nYou are a specialized ECHR legal expert tasked with determining if a case is a key case in ECHR jurisprudence. Your role is to conduct an exhaustive legal analysis, akin to writing a detailed case commentary.\nYour task is to classify the case at /Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-202621.json into one of the following categories:\n\nKEY CASE:\n- Makes a significant contribution to the development, clarification, or modification of case law.\n- Establishes new legal principles or substantially modifies existing ones.\n- Has broad implications beyond the immediate case.\n\nNOT KEY CASE:\n- Applies existing case law without significant contributions to legal development.\n- Demonstrates limited implications beyond the immediate dispute.\n\n\nANALYSIS WORKFLOW:\n\n-Extract Case Facts and Law\n    - Load the case facts and law from the file to understand the context and background.\n\n- Identify Core Legal Issues\n    - Determine the key legal questions and ass

In [9]:
case_path = data_path + df1[45]
case_path

'/Users/ahmed/Desktop/msc-24/ECHR/echr-processed/001-202621.json'